<a href="https://colab.research.google.com/github/andrewcgaitskell/colab-threejs-charting/blob/main/quart_threejs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quart + Jinja2 + Three.js Chart Demo

This notebook demonstrates how to:
- Use **Pandas** to create sample data
- Build an async web server with **Quart**
- Render HTML templates with **Jinja2**
- Visualize data with **Three.js** 3D charts

**Goal**: Show a simple, clean pattern for building interactive 3D visualizations.


## 1. Install Dependencies

Install Quart (async web framework) and required packages.


In [1]:
!pip install -q quart pandas numpy requests


## 2. Create Sample Data

Generate simple sales data using Pandas.

In [2]:
import pandas as pd
import numpy as np

# Create sample quarterly sales data for different products
data = {
    'Product': ['Product A', 'Product B', 'Product C', 'Product D'],
    'Q1': [45, 60, 38, 52],
    'Q2': [58, 72, 45, 61],
    'Q3': [62, 68, 51, 58],
    'Q4': [71, 85, 59, 67]
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df)

# Convert to format suitable for JavaScript
chart_data = df.to_dict('records')
print("\nData for chart:", chart_data)

Sample Data:
     Product  Q1  Q2  Q3  Q4
0  Product A  45  58  62  71
1  Product B  60  72  68  85
2  Product C  38  45  51  59
3  Product D  52  61  58  67

Data for chart: [{'Product': 'Product A', 'Q1': 45, 'Q2': 58, 'Q3': 62, 'Q4': 71}, {'Product': 'Product B', 'Q1': 60, 'Q2': 72, 'Q3': 68, 'Q4': 85}, {'Product': 'Product C', 'Q1': 38, 'Q2': 45, 'Q3': 51, 'Q4': 59}, {'Product': 'Product D', 'Q1': 52, 'Q2': 61, 'Q3': 58, 'Q4': 67}]


In [3]:
import os
import requests

# Create directories
os.makedirs('/content/three/build', exist_ok=True)
os.makedirs('/content/three/examples/jsm/controls', exist_ok=True)

# Three.js files to download
files_to_download = {
    'three.module.js': 'https://cdn.jsdelivr.net/npm/three@0.160.0/build/three.module.js',
    'OrbitControls.js': 'https://cdn.jsdelivr.net/npm/three@0.160.0/examples/jsm/controls/OrbitControls.js',
}

# Download main Three.js library
print("📥 Downloading Three.js libraries...")
response = requests.get(files_to_download['three.module.js'])
if response.status_code == 200:
    with open('/content/three/build/three.module.js', 'w') as f:
        f.write(response.text)
    print("✅ three.module.js downloaded")
else:
    print(f"❌ Failed to download three.module.js: {response.status_code}")

# Download OrbitControls
response = requests.get(files_to_download['OrbitControls.js'])
if response.status_code == 200:
    with open('/content/three/examples/jsm/controls/OrbitControls.js', 'w') as f:
        f.write(response.text)
    print("✅ OrbitControls.js downloaded")
else:
    print(f"❌ Failed to download OrbitControls.js: {response.status_code}")

# Verify downloads
print("\n📁 Downloaded files:")
!ls -lh /content/three/build/
!ls -lh /content/three/examples/jsm/controls/

📥 Downloading Three.js libraries...
✅ three.module.js downloaded
✅ OrbitControls.js downloaded

📁 Downloaded files:
total 1.3M
-rw-r--r-- 1 root root 1.3M Feb 14 15:54 three.module.js
total 32K
-rw-r--r-- 1 root root 30K Feb 14 15:54 OrbitControls.js


## 3. Set Up Quart Application

Create a Quart app with a single route that renders our chart.

In [4]:
from quart import Quart, render_template
import json
import requests
import os

# Setup template directory
TEMPLATE_DIR = '/content/templates'
os.makedirs(TEMPLATE_DIR, exist_ok=True)

# Download template
TEMPLATE_PATH = os.path.join(TEMPLATE_DIR, 'chart_template.html')
TEMPLATE_URL = 'https://raw.githubusercontent.com/andrewcgaitskell/colab-threejs-charting/main/templates/chart_template.html'

if not os.path.exists(TEMPLATE_PATH):
    print("📥 Downloading template...")
    response = requests.get(TEMPLATE_URL, timeout=10)
    if response.status_code == 200:
        with open(TEMPLATE_PATH, 'w') as f:
            f.write(response.text)
        print(f"✅ Template saved to {TEMPLATE_PATH}")
else:
    print(f"✅ Template already exists at {TEMPLATE_PATH}")

# Initialize Quart with template folder
app = Quart(__name__, template_folder=TEMPLATE_DIR)

@app.route('/')
async def index():
    """Main route that renders the 3D chart with data"""
    return await render_template('chart_template.html', data=chart_data)

print("✓ Quart app configured")


✅ Template already exists at /content/templates/chart_template.html
✓ Quart app configured


## 4. Run the Server

Start the Quart server. In Colab, the server will run on localhost.


In [ ]:
import asyncio
from google.colab import output

# Enable port forwarding and display in iframe
output.serve_kernel_port_as_iframe(5000, height=600)

# Run the Quart app
await app.run_task(host='0.0.0.0', port=5000)


<IPython.core.display.Javascript object>

[2026-02-14 15:54:57 +0000] [4037] [INFO] Running on http://0.0.0.0:5000 (CTRL + C to quit)
INFO:hypercorn.error:Running on http://0.0.0.0:5000 (CTRL + C to quit)


[2026-02-14 15:54:57 +0000] [4037] [INFO] 127.0.0.1:45798 GET / 1.1 200 6060 8433


---

## Summary

This notebook demonstrates:

1. **Data Pipeline**: Pandas → Python dict → JSON → JavaScript
2. **Web Framework**: Quart (async) with Jinja2 templating
3. **3D Visualization**: Three.js for interactive charts

### Key Features:
- ✅ Simple, minimal code
- ✅ Interactive 3D controls (drag to rotate, scroll to zoom)
- ✅ Clean data flow from Python to JavaScript
- ✅ Professional presentation-ready output

### Customization Ideas:
- Change the data in Step 2
- Modify colors in the Three.js code
- Add more chart types (scatter, line, etc.)
- Enhance with additional Three.js features (labels, animations, etc.)
